# Successful PDF Scraper

---

## To Run, change `directory` to the folder where all your PDFs are stored. This folder MUST contain nothing else but the files from which you will extact data. 

In [33]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import pdfplumber
import csv
import sys 
import os
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
# From PDFInterpreter import both PDFResourceManager and PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
# Import this to raise exception whenever text extraction from PDF is not allowed
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.layout import LAParams, LTTextBox, LTTextLine
from pdfminer.converter import PDFPageAggregator
import nltk
from nltk.tokenize import RegexpTokenizer
import pandas as pd

directory = r'C:\Users\mered\Documents\01 Current Projects\Marty'
df = pd.DataFrame(columns= ['adrenalMorning', 'adrenalAfternoon', 'adrenalEvening', 'adrenalNight', 'cortisol', 'DHEA', 
                            'insulinFasting', 'insulinNonFasting', 'progesterone', 'SIGA', 'gliadin'])
for filename in os.scandir(directory):
    password = ""
    extracted_text = ""
    c=0
    d = 0
    e = 0
    f = 0
    g = 0
    h = 0
    i = 0
    empty_list = []
    cortisol_list = " "
    DHEA_list = " "
    insulin_list = []
    progesterone_list = " "
    SIGA_list = " "
    gliadin_list = " "
    if filename.path.endswith(".pdf"):
        log_file = os.path.join(directory + "/" + "pdf_log.txt")
        # Open and read the pdf file in binary mode
        fp = open(filename, "rb")
        #print(fp)
        # Create parser object to parse the pdf content
        parser = PDFParser(fp)
        # Store the parsed content in PDFDocument object
        document = PDFDocument(parser, password)
        #print(document)
        # Check if document is extractable, if not abort
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        # Create PDFResourceManager object that stores shared resources such as fonts or images
        rsrcmgr = PDFResourceManager()
        # set parameters for analysis
        laparams = LAParams()
        # Create a PDFDevice object which translates interpreted information into desired format
        # Device needs to be connected to resource manager to store shared resources
        # device = PDFDevice(rsrcmgr)
        # Extract the decive to page aggregator to get LT object elements
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        # Create interpreter object to process page content from PDFDocument
        # Interpreter needs to be connected to resource manager for shared resources and device 
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        # Ok now that we have everything to process a pdf document, lets process it page by page
        for page in PDFPage.create_pages(document):
            # As the interpreter processes the page stored in PDFDocument object
            interpreter.process_page(page)
            # The device renders the layout from interpreter
            layout = device.get_result()
            # Out of the many LT objects within layout, we are interested in LTTextBox and LTTextLine
            for lt_obj in layout:
                if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
                    extracted_text += lt_obj.get_text()
        #close the pdf file
        fp.close()
        #print (extracted_text.encode("utf-8"))
                
        with open(log_file, "wb") as my_log:
            my_log.write(extracted_text.encode("utf-8"))
        #print("Done !!")
            tokenizer = RegexpTokenizer('\w+')
            tokens = tokenizer.tokenize(extracted_text)
            #print(tokens)
            for word in tokens:
                if word == "Midnight":
                    c = c + 1
                    new_word = tokens[c]
                    empty_list.append(new_word)
                    c = c + 2
                    new_word = tokens[c]
                    empty_list.append(new_word)
                    c = c + 2
                    new_word = tokens[c]
                    empty_list.append(new_word)
                    c = c + 2
                    new_word = tokens[c]
                    empty_list.append(new_word)
                else: 
                    c = c + 1
                if word == "Circadian":
                    d = d + 48
                    new_word2 = tokens[d]
                    cortisol_list = new_word2
                else:
                    d = d + 1
                if word =="Dehydroepiandrosterone":
                    e = e + 7
                    new_word3 = tokens[e]
                    DHEA_list = new_word3
                else: 
                    e = e + 1
                if word == "ISN":
                    f = f + 17
                    new_word4 = tokens[f]
                    insulin_list.append(new_word4)
                    f = f + 1
                    new_word4 = tokens[f]
                    insulin_list.append(new_word4)
                else:
                    f = f + 1
                if word == "Progesterone":
                    g = g + 1
                    new_word5 = tokens[g]
                    progesterone_list = new_word5
                else:
                    g = g + 1
                if word == "MB2S":
                    h = h + 4
                    new_word6 = tokens[h]
                    SIGA_list = new_word6
                else: 
                    h = h + 1
                if word == "FI4":
                    i = i + 5
                    new_word7 = tokens[i]
                    gliadin_list = new_word7
                else: 
                    i = i + 1

            adrenal_values = empty_list[0:4]
            adrenalMorning = adrenal_values[0]
            adrenalAfternoon = adrenal_values[1]
            adrenalEvening = adrenal_values[2]
            adrenalNight = adrenal_values[3]
            insulinFasting = insulin_list[0]
            insulinNonFasting = insulin_list[1]
            #print(adrenalMorning)
            #print(adrenalAfternoon)
            #print(adrenalEvening)
            #print(adrenalNight)
            #print(cortisol_list)
            #print(DHEA_list)
            #print(insulinFasting)
            #print(insulinNonFasting)
            #print(progesterone_list)
            #print(SIGA_list)
            #print(gliadin_list)
            df = df.append({'adrenalMorning': adrenalMorning, 'adrenalAfternoon': adrenalAfternoon, 'adrenalEvening' : adrenalEvening, 
                        'adrenalNight' : adrenalNight, 'cortisol' : cortisol_list, 'DHEA' : DHEA_list, 
                        'insulinFasting' : insulinFasting, 'insulinNonFasting' : insulinNonFasting, 
                        'progesterone' : progesterone_list, 'SIGA' : SIGA_list, 'gliadin' : gliadin_list}, ignore_index = True)
df.head()

['Age', 'DOB', 'Specimen', 'Collected', '08', '20', '2013', 'Tel', '425', '251', '0596', 'CLIA', 'License', '50D0630141', '3', 'USA', 'Tel', '916', '448', '9927', 'Fax', '1', '916', '448', '8165', 'Test', 'Description', 'Result', 'Ref', 'Values', 'ASI', 'Adrenal', 'Stress', 'Index', 'Original', 'Saliva', 'Figure', '1', 'Circadian', 'Cortisol', 'Profile', 'TAP', 'Free', 'Cortisol', 'Rhythm', 'Saliva', '06', '00', '08', '00', 'AM', '11', '00', '1', '00', 'PM', '04', '00', '05', '00', 'PM', '10', '00', 'Midnight', '17', 'Normal', '5', 'Normal', '4', 'Normal', '1', 'Normal', '13', '24', 'nM', '5', '10', 'nM', '3', '8', 'nM', '1', '4', 'nM', 'Cortisol', 'Load', '27', '22', '46', 'nM', 'The', 'cortisol', 'load', 'reflects', 'the', 'area', 'under', 'the', 'cortisol', 'curve', 'This', 'is', 'an', 'indicator', 'of', 'overall', 'cortisol', 'exposure', 'where', 'high', 'values', 'favor', 'a', 'catabolic', 'state', 'and', 'low', 'values', 'are', 'sign', 'of', 'adrenal', 'deterioration', 'Figure', 

['Age', 'DOB', 'Specimen', 'Collected', '08', '28', '2013', 'Test', 'Description', 'Result', 'Ref', 'Values', 'ASI', 'Adrenal', 'Stress', 'Index', 'Original', 'Saliva', 'Figure', '1', 'Circadian', 'Cortisol', 'Profile', 'TAP', 'Free', 'Cortisol', 'Rhythm', 'Saliva', '06', '00', '08', '00', 'AM', '11', '00', '1', '00', 'PM', '04', '00', '05', '00', 'PM', '10', '00', 'Midnight', '29', 'Elevated', '5', 'Normal', '5', 'Normal', '7', 'Elevated', '13', '24', 'nM', '5', '10', 'nM', '3', '8', 'nM', '1', '4', 'nM', 'Cortisol', 'Load', '46', '22', '46', 'nM', 'The', 'cortisol', 'load', 'reflects', 'the', 'area', 'under', 'the', 'cortisol', 'curve', 'This', 'is', 'an', 'indicator', 'of', 'overall', 'cortisol', 'exposure', 'where', 'high', 'values', 'favor', 'a', 'catabolic', 'state', 'and', 'low', 'values', 'are', 'sign', 'of', 'adrenal', 'deterioration', '30', '25', '20', '15', '10', '5', '0', '8', 'AM', 'Noon', '4', 'PM', 'Reference', 'Ranges', 'Midnight', 'Patient', 'Results', 'Figure', '2', '

,adrenalMorning,adrenalAfternoon,adrenalEvening,adrenalNight,cortisol,DHEA,insulinFasting,insulinNonFasting,progesterone,SIGA,gliadin
0,17,5,4,1,27,5,3,3,94,10,1
1,30,5,4,1,40,5,3,3,55,15,2
2,29,5,5,7,46,3,3,3,60,14,26
